In [1]:
import numpy as np
from collections import Counter

In [2]:

# Step 1: Entropy
def entropy(y):
    counter = Counter(y)
    total = len(y)
    ent = 0.0
    for count in counter.values():
        p = count / total
        ent -= p * np.log2(p)
    return ent

In [3]:
# Step 2: Information Gain
def information_gain(X_column, y):
    parent_entropy = entropy(y)
    values, counts = np.unique(X_column, return_counts=True)
    
    weighted_entropy = 0.0
    for val, count in zip(values, counts):
        subset_y = y[X_column == val]
        weighted_entropy += (count / len(X_column)) * entropy(subset_y)
    
    return parent_entropy - weighted_entropy

In [4]:
# Step 3: ID3 Recursive Tree
class Node:
    def __init__(self, feature=None, value=None, left=None, right=None, *, label=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.label = label

In [5]:
def id3(X, y, features):
    # If pure, return leaf node
    if len(set(y)) == 1:
        return Node(label=y[0])

    # If no features left, return majority
    if len(features) == 0:
        most_common_label = Counter(y).most_common(1)[0][0]
        return Node(label=most_common_label)

    # Find best feature to split
    gains = [information_gain(X[:, i], y) for i in features]
    best_idx = features[np.argmax(gains)]
    
    node = Node(feature=best_idx)

    # For each value of the best feature
    feature_values = np.unique(X[:, best_idx])
    if len(feature_values) != 2:
        raise Exception("Simple ID3 assumes binary features for simplicity.")
    
    left_indices = X[:, best_idx] == feature_values[0]
    right_indices = X[:, best_idx] == feature_values[1]

    left_subtree = id3(X[left_indices], y[left_indices], [f for f in features if f != best_idx])
    right_subtree = id3(X[right_indices], y[right_indices], [f for f in features if f != best_idx])

    node.value = feature_values[0]  # Value for left child
    node.left = left_subtree
    node.right = right_subtree

    return node

In [6]:

# Step 4: Prediction
def predict(x, tree):
    while tree.label is None:
        if x[tree.feature] == tree.value:
            tree = tree.left
        else:
            tree = tree.right
    return tree.label


In [8]:
# Small toy dataset
X = np.array([
    [0, 1],
    [0, 0],
    [1, 1],
    [1, 0],
])
y = np.array(['yes', 'no', 'yes', 'no'])

features = list(range(X.shape[1]))

tree = id3(X, y, features)

# Predict
for sample in X:
    print(predict(sample, tree))


yes
no
yes
no
